# Phase kickback

Consider a unitary gate $U$ and its eigenstate $\lvert\psi\rangle$.  
From the nature of unitary gates, the eigenvalues can be expressed as $e^{i\theta}$ using a certain phase angle $\theta$.

$$
U\lvert\psi\rangle = e^{i \theta} \lvert\psi\rangle
$$

Let's say we want to find this $\theta$. Since it appears as a global phase, it is difficult to measure it directly.  

Phase kickback is a technique for extracting $\theta$ information into a different qubit than $\lvert\psi\rangle$.  
Then, we can know $\theta$ by measuring the qubit.  
For this purpose, we use $CU$, which is a controlled gate of $U$.

Let's consider a simple example.

First, set the unitary gate as follows, 

$$
U(\theta) = \begin{pmatrix}
1 & 0 \\
0 & e^{i\theta} \\
\end{pmatrix}
$$

In this case, $\lvert 1\rangle$ is an eigenstate. The eigenvalue is $e^{i\theta}$.

$$
U(\theta)\lvert 1 \rangle = e^{i\theta}\lvert 1 \rangle
$$

The control unitary gate is
$$
CU(\theta) = CR(\theta) = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{i\theta} \\
\end{pmatrix}
$$

The phase kickback uses the following quantum circuit.  

<img src="../tutorial-ja/img/120_img.png" width="40%">

First, we create a superposition state by acting an $H$ gate on the auxiliary qubit.

$$
\lvert\psi_1\rangle = \frac{1}{\sqrt{2}}(\lvert0\rangle + \lvert1\rangle) \otimes \lvert1\rangle
$$

Next, let $U(\theta)$ act on the eigenstate with the auxiliary qubit as the control qubit.  
This allows the eigenvalue to be retrieved as a coefficient.

$$
\begin{align}
\lvert\psi_2\rangle &= \frac{1}{\sqrt{2}}(\lvert0\rangle \otimes \lvert1\rangle + \lvert1\rangle \otimes U(\theta)\lvert1\rangle) \\
&= \frac{1}{\sqrt{2}}(\lvert0\rangle \otimes \lvert1\rangle + \lvert1\rangle \otimes e^{i\theta}\lvert1\rangle) \\
&= \frac{1}{\sqrt{2}}(\lvert0\rangle \otimes \lvert1\rangle + e^{i\theta}\lvert1\rangle \otimes \lvert1\rangle) \\
&= \frac{1}{\sqrt{2}}(\lvert0\rangle + e^{i\theta} \lvert1\rangle) \otimes \lvert1\rangle)
\end{align}
$$

As shown above, we have extracted $\theta$ as the coefficient of the superposition state of the auxiliary qubit.  

Here we will focus only on the auxiliary qubit.  
If we measure the auxiliary qubit, $\lvert0\rangle$ and $\lvert1\rangle$ are equally probable, so we will act $H$ gate again.

$$
\begin{align}
H\cdot \frac{1}{\sqrt{2}}(\lvert0\rangle + e^{i\theta} \lvert1\rangle) &= \frac{1}{\sqrt{2}} 
\begin{pmatrix} 1 & 1
\\ 1 & -1 \\
\end{pmatrix}
\cdot
\frac{1}{\sqrt{2}}
\begin{pmatrix} 
1 \\ e^{i\theta} \\ 
\end{pmatrix} \\
&= \frac{1}{2} 
\begin{pmatrix} 
1 + e^{i\theta} \\ 1 - e^{i\theta} \\ 
\end{pmatrix}
\end{align}
$$

We were able to convert the relative phase difference to an amplitude difference using an $H$ gate.   
Finally, if we repeatedly measure the auxiliary qubit, the probability of measuring $\lvert0\rangle$, $\rm{Pr}(\lvert0\rangle)$, is

$$
\begin{align}
\mathrm{Pr}(\lvert0\rangle) &= \biggl| \frac{1+e^{i\theta}}{2} \biggr|^2 \\
&= \frac{1}{2}(1 + \cos\theta)
\end{align}
$$

From the above, we can estimate the real part of the eigenvalue.

Let's implement the above in blueqat.

In [2]:
from blueqat import Circuit
import numpy as np

Implement the above quantum circuit.  
The $\theta$ is determined by a random number.

In [3]:
theta = np.random.rand() * np.pi
shots = 1024

c = Circuit(2)

c.x[1].h[0].cr(theta)[0, 1].h[0].m[0]
res = c.run(shots = shots)

From the results of the run, determine the percentage of $\lvert 0\rangle$ measured on the auxiliary qubit.  
Compare the estimated value $\theta_{est}$ calculated from it with $\theta$ determined in advance by a random number.

In [4]:
p0 = res['00'] / shots
theta_est = np.arccos(2 * p0 - 1)

In [5]:
print("estimated θ　 :", theta_est)
print("actual θ :", theta)

estimated θ　 : 0.8931602777877266
actual θ : 0.8280760567603692


From the above, we were able to estimate $\theta$ by phase kickback.  
The accuracy of the estimation depends on the number of shots.